## 논의하고 싶은 점
1. 코드가 돌아가는 것 같긴 한데.. GPU가 항상 터짐. CPU로 돌리면 느릴 뿐더러 컴퓨터에서 굉음이 남..
2. length가 512보다 크면 처리할 수 없음 -> 그냥 제외? 문단을 잘라서?

코드 출처: https://github.com/donghyeonk/movie-sentiment-analysis-kobert

In [0]:
from datetime import datetime
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
import math
from gluonnlp.data import SentencepieceTokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer
from pytorch_transformers import AdamW, WarmupLinearSchedule  # https://pypi.org/project/pytorch-transformers/

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


### KoBERT

In [0]:
# load pretrained KoBERT
model, vocab = get_pytorch_kobert_model(ctx='cpu')#'cuda' if torch.cuda.is_available() else 'cpu')

using cached model
using cached model


In [0]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [0]:
# get sentencepiece tokenizer
tok_path = get_tokenizer()
sp = SentencepieceTokenizer(tok_path)

using cached model


### torch

In [0]:
class MBTIDataset(Dataset):
    def __init__(self, examples):
        self.examples = examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, index):
        return self.examples[index]

In [0]:
def get_data(filepath, vocab, sp, max_seq_len):
    data = list()
    with open(filepath, 'r', encoding='utf-8') as f:
        for lidx, l in enumerate(f):
            if 0 == lidx:  # header는 제외
                continue
            cols = l[:-1].split('\t') # 마지막 \n 제외하고 \t를 기준으로 분리하기
            doc = cols[0]
            label = cols[1]

            if len(doc) > max_seq_len - 2:  # [CLS]와 [SEP] 빼고
                doc = doc[0:(max_seq_len - 2)]
            
            token_ids = list()
            token_ids.append(vocab['[CLS]'])  # 문장의 시작
            for t in sp(doc):
                if t in vocab: # vocab에 있는 단어라면 해당 단어의 index 찾아서 넣어주기
                    token_ids.append(vocab[t])
                else:          # vocab에 없다면 UNK로 처리
                    token_ids.append(vocab['[UNK]'])
            token_ids.append(vocab['[SEP]']) # 문장의 끝

            data.append([token_ids, int(label)])

    return data

In [0]:
def batchify(b):
    x_len = [len(e[0]) for e in b] # 각 doc의 길이
    batch_max_len = max(x_len) # 제일 긴 doc의 길이

    x = list()
    tk_type_ids = list()
    x_mask = list()
    y = list()
    for e in b:
        seq_len = len(e[0])
        e0_mask = [1] * seq_len  # 1: MASK 
        while len(e[0]) < batch_max_len:  # 제일 긴 doc의 길이에 맞추어 [PAD]로 채워주기
            e[0].append(0)  # 0: '[PAD]'
            e0_mask.append(0)
        assert len(e[0]) == batch_max_len

        e0_tk_type_ids = [0] * batch_max_len
        # e0_tk_type_ids[seq_len - 1] = 1

        x.append(e[0])
        tk_type_ids.append(e0_tk_type_ids)
        x_mask.append(e0_mask)
        y.append(e[1])

    x = torch.tensor(x, dtype=torch.int64)
    tk_type_ids = torch.tensor(tk_type_ids, dtype=torch.int64)
    x_mask = torch.tensor(x_mask, dtype=torch.int64)
    y = torch.tensor(y, dtype=torch.int64)

    return x, tk_type_ids, x_mask, y

In [0]:
data = get_data('train_1.txt', vocab, sp, max_seq_len=256)

In [0]:
len(data)

22511

In [0]:
# torch setting
lr = 5e-5
batch_size = 16
epochs = 5
dropout_rate = 0.1
max_grad_norm = 1.0
num_total_steps = math.ceil(len(data) / batch_size) * epochs
num_warmup_steps = num_total_steps // 10
log_interval = 100
seed = 2019
num_workers = 0 #2  # 자꾸 에러 나서 일단 0으로 설정해놨음
num_classes = 2
pooler_out_dim = model.pooler.dense.out_features

torch.manual_seed(seed)

device = torch.device("cpu")#"cuda" if torch.cuda.is_available() else "cpu")
print('device', device)

device cpu


In [0]:
train_loader = torch.utils.data.DataLoader(
    MBTIDataset(data[:10]),  # 일단 10개만으로 시도
    shuffle=True,
    batch_size=batch_size,
    num_workers=num_workers,
    collate_fn=batchify,
    pin_memory=True  # GPU에 필요한 argument인듯
)

In [0]:
linear = torch.nn.Linear(pooler_out_dim, num_classes).to(device)  # last layer for classification

all_params = list(model.parameters()) + list(linear.parameters())
optimizer = AdamW(all_params, lr=lr, correct_bias=False)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=num_warmup_steps,
                                 t_total=num_total_steps)

In [0]:
def train(train_loader, valid_loader, device, model, linear, all_params, optimizer, scheduler,
      dropout_rate, max_grad_norm, log_interval, n_epochs=20, save_file_name, max_epochs_stop=3, print_every=1):

    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    for epoch in range(n_epochs):
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        model.train()
        linear.train()
        start = timer()

        for batch_idx, (input_ids, token_type_ids, input_mask, target) in enumerate(train_loader):
            if cuda.is_available():
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                input_mask = input_mask.to(device)
                target = target.to(device)

            optimizer.zero_grad()
            _, pooled_output = model(input_ids, token_type_ids, input_mask)
            logits = linear(F.dropout(pooled_output, p=dropout_rate))
            output = F.log_softmax(logits, dim=1)

            loss = F.nll_loss(output, target)  # negative log-likelihood
            loss.backward()
            torch.nn.utils.clip_grad_norm_(all_params, max_grad_norm) # exploding gradient 방지
            optimizer.step()
            scheduler.step()

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * data.size(0)

            _, pred = output.argmax(dim=1, keepdim=True)
            correct_tensor = pred.eq(target.view_as(pred)).sum().item()
            #correct_tensor = pred.eq(target.data.view_as(pred))

            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))

            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)

        # Track training progress
        print(f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. 
              {timer() - start:.2f} seconds elapsed in epoch.',end='\r')

      # After training loops ends, start validation      
      else:
        model.epochs += 1

        with torch.no_grad():
          model.eval()

          for data, target in valid_loader:
            if train_on_gpu:
              input_ids = input_ids.to(device)
              token_type_ids = token_type_ids.to(device)
              input_mask = input_mask.to(device)
              target = target.to(device)

            _, pooled_output = model(input_ids, token_type_ids, input_mask)
            logits = linear(F.dropout(pooled_output, p=dropout_rate))
            output = F.log_softmax(logits, dim=1)

            # Validation loss
            loss = F.nll_loss(output, target)  # negative log-likelihood

            # Multiply average loss times the number of examples in batch
            valid_loss += loss.item() * data.size(0)

            _, pred = output.argmax(dim=1, keepdim=True)
            correct_tensor = pred.eq(target.view_as(pred)).sum().item()
            #correct_tensor = pred.eq(target.data.view_as(pred))

            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))

            # Multiply average accuracy times the number of examples in batch
            valid_acc += accuracy.item() * data.size(0)            

        # Calculate average losses
        train_loss = train_loss / len(train_loader.dataset)
        valid_loss = valid_loss / len(valid_loader.dataset)

        # Calculate average accuracy
        train_acc = train_acc / len(train_loader.dataset)
        valid_acc = valid_acc / len(valid_loader.dataset)

        history.append([train_loss, valid_loss, train_acc, valid_acc])

        if (epoch + 1) % print_every == 0:
          print('\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}')
          print(f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%')

        if valid_loss < valid_loss_min:
          # Save model
          torch.save(model.state_dict(), save_file_name)
          # Track improvement
          epochs_no_improve = 0
          valid_loss_min = valid_loss
          valid_best_acc = valid_acc
          best_epoch = epoch

        # Otherwise increment count of epochs with no improvement
        else:
          epochs_no_improve += 1
          # Trigger early stopping
          if epochs_no_improve >= max_epochs_stop:
            print(f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%')
            total_time = timer() - overall_start
            print(f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.')

            # Load the best state dict
            model.load_state_dict(torch.load(save_file_name))
            # Attach the optimizer
            model.optimizer = optimizer
            # Format history
            history = pd.DataFrame(history, columns=['train_loss', 'valid_loss', 'train_acc','valid_acc'])
            return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%')
    print(f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.')

    # Format history
    history = pd.DataFrame(history,columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])

    return model, history  